## # Introduction
<p><img src="https://i.imgur.com/kjWF1So.jpg" alt="Different characters on a computer screen"></p>
<p>According to a 2019 <a href="https://storage.googleapis.com/gweb-uniblog-publish-prod/documents/PasswordCheckup-HarrisPoll-InfographicFINAL.pdf">Google / Harris Poll</a>, 24% of Americans have used common passwords, like <code>abc123</code>, <code>Password</code>, and <code>Admin</code>. Even more concerning, 59% of Americans have incorporated personal information, such as their name or birthday, into their password. This makes it unsurprising that 4 in 10 Americans have had their personal information compromised online. Passwords with commonly used phrases and personal information makes cracking a password drastically easier.</p>
<p>You may have noticed over the years that password requirements have increased in complexity, including recommendations to change your passwords every couple of months. Compiled from industry recommendations, below is a list of passwords requirements you will be asked to test: </p>
<p><strong>Password Requirments:</strong></p>
<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>
<p>Here is the dataset that you will investigate this project:</p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/logins.csv</b></div>
Each row represents a login credential. There are no missing values and you can consider the dataset "clean".
<ul>
    <li><b>id:</b> the user's unique ID.</li>
    <li><b>username:</b> the username with the format {firstname}.{lastname}.</li>
    <li><b>password:</b> the password that may or may not meet the requirements. <i>Note, passwords should never be saved in plaintext, always encrypt them when working with real live passwords!</i></li>
</ul>
</div>
<p>Warning: This dataset contains some <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have been filtered, but may still include words that are explicit and offensive.</p>
<p>From here on out, it will be your task to explore and manipulate the existing data until you can answer the two questions described in the instructions panel. Feel free to import as many packages as you need to complete your task, and add cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><strong>Note:</strong> To complete this project, you need to know how to manipulate strings in pandas DataFrames and be familiar with regular expressions. Before starting this project we recommend that you have completed the following courses: <a href="https://learn.datacamp.com/courses/data-cleaning-in-python">Data Cleaning in Python</a> and <a href="https://learn.datacamp.com/courses/regular-expressions-in-python">Regular Expressions in Python</a>.</p>

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("datasets/logins.csv")

In [3]:
df.head()

,id,username,password
0,1,vance.jennings,vanceRules888!
1,2,consuelo.eaton,Mail_Pen%Scarlets.414
2,3,mitchel.perkins,Z00+1960
3,4,odessa.vaughan,D-rockyou
4,5,araceli.wilder,Araceli}r3


In [4]:
# Longitud minima de 10 
pws_no_validos = df[~(df['password'].str.len()>= 10)]
pws_validos = df[(df['password'].str.len()>= 10)]

In [5]:
pws_no_validos.head()

,id,username,password
2,3,mitchel.perkins,Z00+1960
3,4,odessa.vaughan,D-rockyou
9,10,brant.zimmerman,L?4)OSB$r
16,17,domingo.dyer,VeOw{*p
17,18,martin.pacheco,MP1985???


In [6]:
#Debe contener: Minusculas, mayusculas, digitos, caracteres alphanumericos
minus_case = pws_validos['password'].str.contains('[a-z]')
mayus_case = pws_validos['password'].str.contains('[A-Z]')
num_case = pws_validos['password'].str.contains('\d')
especial_case = pws_validos['password'].str.contains('\W')

In [7]:
# Deber contener todas las reglas de la celda anterior es decir la union de ellos igual a True
char_check = minus_case & mayus_case & num_case & especial_case
#Se agrega al df de password incorrectas los que no cumplen con la condicion de char_check
pws_no_validos = pws_no_validos.append(pws_validos[~char_check], ignore_index=True)
#Se quitan los que no cumplieron la condicion
pws_validos =  pws_validos[char_check]

In [8]:
pws_validos.head()

,id,username,password
0,1,vance.jennings,vanceRules888!
1,2,consuelo.eaton,Mail_Pen%Scarlets.414
4,5,araceli.wilder,Araceli}r3
6,7,evelyn.gay,`4:&iAt$'o~(
8,9,gladys.ward,=Wj1`i)xYYZ


In [9]:
# Pasa a malos los que contienen la palabra password
palabra_prohibida = pws_validos['password'].str.contains('password') 
pws_no_validos = pws_no_validos.append(pws_validos[palabra_prohibida],ignore_index=True)
pws_validos = pws_validos[~palabra_prohibida]

In [10]:
#Crea dos nuevas columnas donde se agrega el nombre y apellido contenidos en el username
pws_validos['first_name'] = pws_validos['username'].str.extract('(^\w+)', expand = False)
pws_validos['last_name'] = pws_validos['username'].str.extract('(\w+$)', expand = False)

#Elimina los que contienen el nombre o apellido de los validos y los agregan a no_validos
for i, row in pws_validos.iterrows():
    if row.first_name in row.password.lower() or row.last_name in row.password.lower():
        pws_validos = pws_validos.drop(index=i)
        pws_no_validos = pws_no_validos.append(row,ignore_index=True)


In [11]:
bad_pass = round(pws_no_validos.shape[0] / df.shape[0], 2)
print("Percentage of users with invalid passwords: ",bad_pass)

Percentage of users with invalid passwords:  0.75


In [12]:
email_list = pws_no_validos['username'].sort_values()
email_list.head()

405    abdul.rowland
309     addie.cherry
372     adele.moreno
517     adeline.bush
279      adolfo.kane
Name: username, dtype: object

In [13]:
email_list.shape[0]

736

**Conclusiones**  
En esta libreta se utilizo el dataset logins.csv para validar las contraseñas de los usuarios. En este dataset vienen 2 columnas: 
- *username*: Contiene el nombre y apellido del usuario con el siguiente formato {nombre}.{apellido}
- *password*: Contiene la contraseña del usuario

Se desarrollaron disstintos procedimientos con la finalidad de contestar dos preguntas:
1. ¿Cual es el porcentaje de usuarios que su contraseña es no valida, es decir, que no cumple con las condiciones de una contraseña segura?
2. ¿Cuales son los usuarios que necesitan cambiar su contraseña?

Para esto se realizo el siguiente procedimiento:
- Se descargó el dataset y se guardó como un dataframe en la variable *df*.
- Se crearon dos dataframes para almacenar las contraseñas válidas (*pws_validos*) y las contraseñas no válidas (*pws_no_validos*), en donde se incluyeron las contraseñas que no cumplen con la longitud mínima requerida de 10 caracteres.
- Se eliminaron las contraseñas del dataframe de contraseñas válidas que no cumplían con los requisitos de contener minúsculas, mayúsculas, dígitos y caracteres especiales, y se agregaron al dataframe de las no válidas.
- Se crearon dos columnas, *first_name* y *last_name*, en las que se almacenaron el nombre y el apellido respectivamente, extrayéndolos de la columna *username*.
- Se eliminaron las contraseñas válidas que contenían el nombre o el apellido en la contraseña y se agregaron al dataframe de las no válidas.
- Se calculó el porcentaje de contraseñas no válidas dividiendo la cantidad de contraseñas no válidas entre el total de datos en df, y se redondeó a 2 decimales.
- Se tomaron todos los datos de la columna *username* del dataframe de las contraseñas no válidas y se ordenaron de forma ascendente.

Como resultado se encontro que el 75% de los usuarios tienen contraseñas no validas, lo que equivale a 736 usuarios a los cuales se les tendra que enviar correo solicitando el cambio de contraseña ya que estas no cumplen con las condiciones requeridas para que sea una contraseña segura.